<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/jvcqp2iy2jlx2b32rmzdt0tx8lvxgzkp.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>RECURRENT NETWORKS and LSTM IN DEEP LEARNING</font></h1>


## Applying Recurrent Neural Networks/LSTM for Language Modelling
Hello and welcome to this part. In this notebook, we will go over the topic of what Language Modelling is and create a Recurrent Neural Network model based on the Long Short-Term Memory unit to train and be benchmarked by the Penn Treebank. By the end of this notebook, you should be able to understand how TensorFlow builds and executes a RNN model for Language Modelling.


--------

## The Objective
By now, you should have an understanding of how Recurrent Networks work -- a specialized model to process sequential data by keeping track of the "state" or context. In this notebook, we go over a TensorFlow code snippet for creating a model focused on **Language Modelling** -- a very relevant task that is the cornerstone of many different linguistic problems such as **Speech Recognition, Machine Translation and Image Captioning**. For this, we will be using the Penn Treebank, which is an often-used dataset for benchmarking Language Modelling models.

## What exactly is Language Modelling?
Language Modelling, to put it simply, **is the task of assigning probabilities to sequences of words**. This means that, given a context of one or a few words in the language the model was trained on, the model should have a knowledge of what are the most probable words or sequence of words for the sentence. Language Modelling is one of the tasks under Natural Language Processing, and one of the most important.

<img src=https://ibm.box.com/shared/static/1d1i5gub6wljby2vani2vzxp0xsph702.png width="768"/>
<center>*Example of a sentence being predicted*</center>

In this example, one can see the predictions for the next word of a sentence, given the context "This is an". As you can see, this boils down to a sequential data analysis task -- you are given a word or a sequence of words (the input data), and, given the context (the state), you need to find out what is the next word (the prediction). This kind of analysis is very important for language-related tasks such as **Speech Recognition, Machine Translation, Image Captioning, Text Correction** and many other very relevant problems. 

<img src=https://ibm.box.com/shared/static/az39idf9ipfdpc5ugifpgxnydelhyf3i.png width="1080"/>
<center>*The above example schematized as an RNN in execution*</center>

As the above image shows, Recurrent Network models fit this problem like a glove. Alongside LSTM and its capacity to maintain the model's state for over one thousand time steps, we have all the tools we need to undertake this problem. The goal for this notebook is to create a model that can reach **low levels of perplexity** on our desired dataset.

For Language Modelling problems, **perplexity** is the way to gauge efficiency. Perplexity is simply a measure of how well a probabilistic model is able to predict its sample. A higher-level way to explain this would be saying that **low perplexity means a higher degree of trust in the predictions the model makes**. Therefore, the lower perplexity is, the better.

## The Penn Treebank dataset
Historically, datasets big enough for Natural Language Processing are hard to come by. This is in part due to the necessity of the sentences to be broken down and tagged with a certain degree of correctness -- or else the models trained on it won't be able to be correct at all. This means that we need a **large amount of data, annotated by or at least corrected by humans**. This is, of course, not an easy task at all.

The Penn Treebank, or PTB for short, is a dataset maintained by the University of Pennsylvania. It is *huge* -- there are over **four million and eight hundred thousand** annotated words in it, all corrected by humans. It is composed of many different sources, from abstracts of Department of Energy papers to texts from the Library of America. Since it is verifiably correct and of such a huge size, the Penn Treebank has been used time and time again as a benchmark dataset for Language Modelling.

The dataset is divided in different kinds of annotations, such as Piece-of-Speech, Syntactic and Semantic skeletons. For this example, we will simply use a sample of clean, non-annotated words (with the exception of one tag -- `<unk>`, which is used for rare words such as uncommon proper nouns) for our model. This means that we just want to predict what the next words would be, not what they mean in context or their classes on a given sentence. 
<br/>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<center>the percentage of lung cancer deaths among the workers at the west `<unk>` mass. paper factory appears to be the highest for any asbestos workers studied in western industrialized countries he said 
 the plant which is owned by `<unk>` & `<unk>` co. was under contract with `<unk>` to make the cigarette filters 
 the finding probably will support those who argue that the u.s. should regulate the class of asbestos including `<unk>` more `<unk>` than the common kind of asbestos `<unk>` found in most schools and other buildings dr. `<unk>` said</center>
</div>
 <center>*Example of text from the dataset we are going to use, `ptb.train`*</center>
 <br/>



<h2>Word Embeddings</h2><br/>

For better processing, in this example, we will make use of [**word embeddings**]( [https://www.tensorflow.org/tutorials/word2vec/), which are **a way of representing sentence structures or words as n-dimensional vectors (where n is a reasonably high number, such as 200 or 500) of real numbers**. Basically, we will assign each word a randomly-initialized vector, and input those into the network to be processed. After a number of iterations, these vectors are expected to assume values that help the network to correctly predict what it needs to -- in our case, the probable next word in the sentence. This is shown to be very effective in Natural Language Processing tasks, and is a commonplace practice.
<br/><br/>
<font size = 4><strong>
$$Vec("Example") = [0.02, 0.00, 0.00, 0.92, 0.30,...]$$
</font></strong>
<br/>
Word Embedding tends to group up similarly used words *reasonably* together in the vectorial space. For example, if we use T-SNE (a dimensional reduction visualization algorithm) to flatten the dimensions of our vectors into a 2-dimensional space and use the words these vectors represent as their labels, we might see something like this:

<img src=https://ibm.box.com/shared/static/bqhc5dg879gcoabzhxra1w8rkg3od1cu.png width="800"/>
<center>*T-SNE Mockup with clusters marked for easier visualization*</center>

As you can see, words that are frequently used together, in place of each other, or in the same places as them tend to be grouped together -- being closer together the higher these correlations are. For example, "None" is pretty semantically close to "Zero", while a phrase that uses "Italy" can probably also fit "Germany" in it, with little damage to the sentence structure. A vectorial "closeness" for similar words like this is a great indicator of a well-built model.

---
 

We need to import the necessary modules for our code. We need **`numpy` and `tensorflow`**, obviously. Additionally, we can import directly the **`tensorflow.models.rnn.rnn`** model, which includes the function for building RNNs, and **`tensorflow.models.rnn.ptb.reader`** which is the helper module for getting the input data from the dataset we just downloaded.

If you want to learm more take a look at https://www.tensorflow.org/versions/r0.11/api_docs/python/rnn_cell/

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import tensorflow as tf
import os

print('TensorFlow version: ', tf.__version__)

TensorFlow version:  1.1.0


In [2]:
tf.reset_default_graph()

In [3]:
if not os.path.isfile('./penn_treebank_reader.py'):
    print('Downloading penn_treebank_reader.py...')
    !wget -q -O ../../data/Penn_Treebank/ptb.zip https://ibm.box.com/shared/static/z2yvmhbskc45xd2a9a4kkn6hg4g4kj5r.zip
    !unzip -o ../../data/Penn_Treebank/ptb.zip -d ../data/Penn_Treebank
    !cp ../../data/Penn_Treebank/ptb/reader.py ./penn_treebank_reader.py
else:
    print('Using local penn_treebank_reader.py...')

Using local penn_treebank_reader.py...


In [4]:
import penn_treebank_reader as reader

## Building the LSTM model for Language Modeling
Now that we know exactly what we are doing, we can start building our model using TensorFlow. The very first thing we need to do is download and extract the `simple-examples` dataset, which can be done by executing the code cell below.

In [5]:
if not os.path.isfile('../data/Penn_Treebank/simple_examples.tgz'):
    !wget -O ../../data/Penn_Treebank/simple_examples.tgz http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
    !tar xzf ../../data/Penn_Treebank/simple_examples.tgz -C ../data/Penn_Treebank/

--2017-05-23 10:45:45--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘../../data/Penn_Treebank/simple_examples.tgz’

../../data/Penn_Tre 100%[===================>]  33,25M  3,79MB/s    in 9,1s    

2017-05-23 10:45:55 (3,67 MB/s) - ‘../../data/Penn_Treebank/simple_examples.tgz’ saved [34869662/34869662]

tar: ../data/Penn_Treebank: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


Additionally, for the sake of making it easy to play around with the model's hyperparameters, we can declare them beforehand. Feel free to change these -- you will see a difference in performance each time you change those!  



In [9]:
#Initial weight scale
init_scale = 0.1
#Initial learning rate
learning_rate = 1.0
#Maximum permissible norm for the gradient (For gradient clipping -- another measure against Exploding Gradients)
max_grad_norm = 5
#The number of layers in our model
num_layers = 2
#The total number of recurrence steps, also known as the number of layers when our RNN is "unfolded"
num_steps = 20
#The number of processing units (neurons) in the hidden layers
hidden_size = 200
#The maximum number of epochs trained with the initial learning rate
max_epoch = 4
#The total number of epochs in training
max_max_epoch = 13
#The probability for keeping data in the Dropout Layer (This is an optimization, but is outside our scope for this notebook!)
#At 1, we ignore the Dropout Layer wrapping.
keep_prob = 1
#The decay for the learning rate
decay = 0.5
#The size for each batch of data
batch_size = 30
#The size of our vocabulary
vocab_size = 10000
#Training flag to separate training from testing
is_training = 1
#Data directory for our dataset
data_dir = "../../data/Penn_Treebank/simple-examples/data/"

Some clarifications for LSTM architecture based on the argumants:

Network structure:
- In this network, the number of LSTM cells are 2. To give the model more expressive power, we can add multiple layers of LSTMs to process the data. The output of the first layer will become the input of the second and so on.
- The recurrence steps is 20, that is, when our RNN is "Unfolded", the recurrence step is 20.   
- the structure is like: 
     - 200 input units -> [200x200] Weight -> 200 Hidden units (first layer) -> [200x200] Weight matrix  -> 200 Hidden units (second layer) ->  [200] weight Matrix -> 200 unit output

Hidden layer:
- Each LSTM has 200 hidden units which is equivalant to the dimensianality of the embedding words and output. 

Input layer: 
- The network has 200 input units. 
- Suppose each word is represented by an embedding vector of dimensionality e=200. The input layer of each cell will have 200 linear units. These e=200 linear units are connected to each of the h=200 LSTM units in the hidden layer (assuming there is only one hidden layer, though our case has 2 layers).
- The input shape is [batch_size, num_steps], that is [30x20]. It will turn into [30x20x200] after embedding, and then 20x[30x200]




There is a lot to be done and a ton of information to process at the same time, so go over this code slowly. It may seem complex at first, but if you try to ally what you just learned about language modelling to the code you see, you should be able to understand it.

This code is adapted from the [PTBModel](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/models/rnn/ptb/ptb_word_lm.py) example bundled with the TensorFlow source code.


#### Train data
The story starts from data: 
- Train data is a list of words, represented by numbers - N=929589 numbers, e.g. [9971, 9972, 9974, 9975,...]
- We read data as mini-batch of size b=30. Assume the size of each sentence is 20 words (num_steps = 20). Then it will take int(N/b*h)+1=1548 iterations for the learner to go through all sentences once. So, the number of iterators is 1548
- Each batch data is read from train dataset of size 600, and shape of [30x20]

First we start an interactive session:

In [10]:
session=tf.InteractiveSession()

In [11]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _ = raw_data

Lets just read one mini-batch now and feed our network:

In [12]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple=next(itera)
x=first_touple[0]
y=first_touple[1]

In [13]:
x.shape

(30, 20)

Lets look at 3 sentences of our input x:

In [14]:
x[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21]], dtype=int32)

In [15]:
size = hidden_size

we define 2 place holders to feed them with mini-batchs, that is x and y:

In [16]:
_input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
_targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

lets defin a dictionary, and use it later to feed the placeholders with our first mini-batch:

In [17]:
feed_dict={_input_data:x, _targets:y}

For example, we can use it to feed _input_data:

In [18]:
session.run(_input_data,feed_dict)

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21],
       [   3,   71,    4,   27,  246,   60,   11,  215,    4,    1, 1846,
           9,    3,   71,  546,    2, 6505,  162,    6,  104],
       [  93,   25,    6,  261,  681,  251,    0,  278, 3246,   13,  200,
           1,    8,  105, 3360,    1,    4,    0,  536,    4],
       [  20,    6,  954,   12,    3,   21,   78,   14,  977,  726,    0,
          37,   42,   34,    5,  437,  116,  206,  927,    2],
       [  18,  296,    7,  201,   76,    4,  182,  560, 3836,   17,  974,
         975,    6,  942,    4,  156,   77, 1570,  288,  644],
       [  23, 1238,  899,    5,   25,  20

In this step, we create the stacked LSTM, which is a 2 layer LSTM network:

In [19]:
stacked_lstm = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(hidden_size, forget_bias=0.0) 
                                            for _ in range(num_layers)]
                                          )

Also, we initialize the states of the nework:

#### _initial_state

For each LCTM, there are 2 state matrics, c_state and m_state.  c_state and m_state represent "Memory State" and "Cell State". Each hidden layer, has a vector of size 30, which keeps the states. so, for 200 hidden units in each LSTM, we have a matrix of size [30x200]

In [20]:
_initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
_initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(30, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(30, 200) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(30, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(30, 200) dtype=float32>))

lets look at the states, though they are all zero for now:

In [21]:
session.run(_initial_state,feed_dict)

(LSTMStateTuple(c=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), h=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)),
 LSTMStateTuple(c=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), h=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., 

### Embeddings
We create the embeddings for our input data. embedding is dictionary of [10000x200] for all 10000 unique words.

In [22]:
try:
    embedding = tf.get_variable("embedding", [vocab_size, hidden_size])  #[10000x200]
except ValueError:
    pass
embedding.get_shape().as_list()

[10000, 200]

In [23]:
session.run(tf.global_variables_initializer())
session.run(embedding, feed_dict)

array([[ 0.00419833, -0.011482  , -0.01549463, ...,  0.01950766,
         0.01832177, -0.02384475],
       [ 0.02171985,  0.00561222,  0.01664338, ...,  0.0237095 ,
        -0.01921788, -0.01106183],
       [ 0.01881259,  0.02292644,  0.00497515, ...,  0.01437099,
         0.01298326, -0.02087402],
       ..., 
       [ 0.01548864,  0.017141  , -0.02069814, ..., -0.01426928,
         0.00501703, -0.02374811],
       [ 0.02391572,  0.01972473, -0.01133233, ...,  0.01748331,
         0.01228622,  0.01979224],
       [ 0.00684312, -0.00510194,  0.01945009, ..., -0.00847306,
         0.02030636, -0.0018806 ]], dtype=float32)

embedding_lookup goes to each row of input_data, and for each word in the row/sentence, finds the correspond vector in embedding.
It creates a [30*20*200] matrix, so, the first elemnt of __inputs__ (the first sentence), is a matrix of 20x200, which each row of it is vector representing a word in the sentence.


In [24]:
# Define where to get the data for our embeddings from
inputs = tf.nn.embedding_lookup(embedding, _input_data)  #shape=(30, 20, 200) 

In [25]:
inputs

<tf.Tensor 'embedding_lookup:0' shape=(30, 20, 200) dtype=float32>

In [26]:
session.run(inputs[0], feed_dict)

array([[-0.01766763, -0.0102601 , -0.00787331, ..., -0.0012469 ,
        -0.01207721, -0.01660829],
       [ 0.02301355,  0.02373759, -0.01453253, ..., -0.02158061,
         0.01211887, -0.01520211],
       [-0.00116363,  0.01608609,  0.00544146, ..., -0.00878267,
         0.00452347,  0.00519271],
       ..., 
       [ 0.01150502,  0.00421331, -0.0161419 , ...,  0.0184783 ,
         0.01090024, -0.01899387],
       [ 0.00922056,  0.01489506, -0.01159351, ..., -0.00680826,
        -0.0197531 ,  0.01292341],
       [ 0.01657515, -0.00247201,  0.02124329, ...,  0.01656852,
        -0.01288021,  0.00498772]], dtype=float32)

### Constructing Recurrent Neural Networks
__tf.nn.dynamicrnn()__ creates a recurrent neural network using __stacked_lstm__ which is an instance of RNNCell. 

The input should be a Tensor of shape: [batch_size, max_time, ...], in our case it would be (30, 20, 200)

This method, returns a pair (outputs, new_state) where:
- outputs is a length T list of outputs (one for each input), or a nested tuple of such elements.
- new_state is the final state


In [27]:
outputs, new_state =  tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=_initial_state)

so, lets look at the outputs. The output of the stackedLSTM comes from 200 hidden_layer, and in each time step(=20), one of them get activated. we use the linear activation to map the 200 hidden layer to a [?x10 matrix]

In [28]:
outputs

<tf.Tensor 'rnn/transpose:0' shape=(30, 20, 200) dtype=float32>

In [29]:
session.run(tf.global_variables_initializer())
session.run(outputs[0], feed_dict)

array([[  2.31600206e-04,  -9.61103651e-05,  -1.28215936e-04, ...,
         -1.00696590e-04,  -1.18866556e-04,   3.85970998e-06],
       [  5.98545128e-04,  -1.57938353e-04,   6.04075285e-05, ...,
          7.42806660e-05,  -4.91725732e-05,   8.44031911e-06],
       [  4.31441877e-04,  -7.12473193e-05,  -2.78066946e-05, ...,
          4.59898321e-04,   2.40637557e-04,  -6.21227009e-05],
       ..., 
       [  4.11826361e-04,  -3.25875997e-04,   2.63570619e-05, ...,
         -5.27737895e-04,   4.79726616e-04,   2.65730941e-05],
       [  6.67331857e-04,  -5.47913078e-04,  -2.34936088e-04, ...,
         -7.87418219e-04,   1.08516018e-03,   4.59716512e-06],
       [  4.38249495e-04,  -5.56082989e-04,  -7.99677218e-04, ...,
         -1.04818272e-03,   1.60882762e-03,  -3.46088491e-04]], dtype=float32)

Lets reshape the output tensor from  [30 x 20 x 200] to [600 x 200]

In [30]:
output = tf.reshape(outputs, [-1, size])
output

<tf.Tensor 'Reshape:0' shape=(600, 200) dtype=float32>

In [31]:
session.run(output[0], feed_dict)

array([  2.31600206e-04,  -9.61103651e-05,  -1.28215936e-04,
         1.82282485e-04,  -1.81498559e-04,   7.17349540e-05,
         2.64149548e-05,   1.84381948e-04,  -1.61701348e-04,
         5.18249755e-04,  -2.19192108e-04,   4.76456655e-04,
        -2.41356611e-04,   7.81506897e-05,   5.45248622e-04,
         1.34541493e-04,  -1.13408685e-04,   3.70240392e-04,
         2.36539927e-04,  -3.05564376e-04,   5.46291849e-05,
        -2.01269737e-04,  -4.69016639e-04,   5.44591494e-05,
         4.91849962e-04,   2.42963564e-04,  -3.79720383e-04,
         5.38949971e-04,  -3.36418016e-04,  -2.48538036e-05,
        -4.22927173e-04,   1.71456923e-04,   3.96906282e-04,
         3.13919358e-04,   6.60799269e-05,  -6.45390013e-04,
         3.86007683e-04,  -8.17814653e-05,  -2.69292592e-04,
        -1.25197927e-04,  -2.99289677e-04,  -3.30634910e-04,
        -2.20897768e-04,  -1.60990108e-04,  -4.80899558e-04,
         9.81728808e-05,   5.63230560e-06,   1.78039350e-04,
        -1.75673253e-04,

### logistic unit
Now, we create a logistic unit to return the probability of the output word. That is, mapping the 600

Softmax = [600 x 200]* [200 x 1000]+ [1 x 1000] -> [600 x 1000]

In [32]:
softmax_w = tf.get_variable("softmax_w", [size, vocab_size]) #[200x1000]
softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
logits = tf.matmul(output, softmax_w) + softmax_b

In [33]:
session.run(tf.global_variables_initializer())
logi = session.run(logits, feed_dict)
logi.shape

(600, 10000)

In [34]:
First_word_output_probablity = logi[0]
First_word_output_probablity.shape

(10000,)

### Prediction
The maximum probablity

In [35]:
embedding_array= session.run(embedding, feed_dict)
np.argmax(First_word_output_probablity)

8647

So, what is the ground truth for the first word of first sentence? 

In [36]:
y[0][0]

9971

Also, you can get it from target tensor, if you want to find the embedding vector:

In [37]:
_targets

<tf.Tensor 'Placeholder_1:0' shape=(30, 20) dtype=int32>

It is time to compare logit with target

In [38]:
targ = session.run(tf.reshape(_targets, [-1]), feed_dict) 

In [39]:
first_word_target_code= targ[0]
first_word_target_code

9971

In [40]:
first_word_target_vec = session.run( tf.nn.embedding_lookup(embedding, targ[0]))
first_word_target_vec

array([  1.86736062e-02,   1.61858462e-02,  -1.40998391e-02,
        -1.36882123e-02,  -1.33983875e-02,   2.05998421e-02,
         2.46733427e-04,  -1.95281785e-02,  -1.66366622e-02,
         2.35252567e-02,  -1.56327747e-02,  -1.79377422e-02,
        -1.52281262e-02,   2.24282108e-02,  -2.25153193e-02,
         1.12179555e-02,   1.13971643e-02,   2.04268061e-02,
        -1.63705237e-02,  -1.23975612e-03,  -1.90807115e-02,
         2.20058970e-02,   1.95904449e-02,  -6.56226464e-03,
        -1.81781966e-02,  -1.43649057e-04,   9.90456715e-03,
         6.35352731e-03,  -1.19924247e-02,   1.18489973e-02,
         2.35010795e-02,   4.13514674e-03,  -9.61622782e-03,
         9.97390598e-03,  -7.55315647e-04,  -1.23191299e-02,
         1.46972574e-02,  -1.02031920e-02,   2.31070556e-02,
         1.71499401e-02,   8.16120580e-03,   8.09521228e-03,
        -1.05001172e-02,  -2.01561172e-02,  -8.07870179e-03,
         5.75733930e-03,  -1.35746440e-02,   9.17498395e-03,
         5.88469952e-03,

#### Objective function

Now we want to define our objective function. Our objective is to minimize loss function, that is, to minimize the average negative log probability of the target words:

loss=−1N∑i=1Nln⁡ptargeti  
This function is already implimented and available in TensorFlow through __sequence_loss_by_example__ so we can just use it here. __sequence_loss_by_example__ is weighted cross-entropy loss for a sequence of logits (per example).  

Its arguments:  

logits: List of 2D Tensors of shape [batch_size x num_decoder_symbols].  
targets: List of 1D batch-sized int32 Tensors of the same length as logits.  
weights: List of 1D batch-sized float-Tensors of the same length as logits.  

In [41]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(_targets, [-1])],[tf.ones([batch_size * num_steps])])

loss is a 1D batch-sized float Tensor [600x1]: The log-perplexity for each sequence.

In [42]:
session.run(loss, feed_dict)

array([ 9.20420647,  9.20577335,  9.19770718,  9.19478989,  9.21243382,
        9.21494293,  9.21947575,  9.22757244,  9.22758675,  9.21636772,
        9.20103645,  9.19449234,  9.22175789,  9.20829582,  9.21987534,
        9.22261238,  9.19444084,  9.19685555,  9.22691154,  9.21609497,
        9.22377777,  9.21007824,  9.20543385,  9.20270443,  9.21428204,
        9.21068382,  9.22520828,  9.20246506,  9.19868851,  9.20911407,
        9.2212925 ,  9.21260929,  9.21115303,  9.21098137,  9.21868134,
        9.21260643,  9.21111774,  9.22663784,  9.19542694,  9.22124577,
        9.21371841,  9.20293427,  9.21089554,  9.20351315,  9.21870041,
        9.22543907,  9.21017075,  9.20417595,  9.20265102,  9.2133112 ,
        9.22036934,  9.19654942,  9.20777988,  9.21099377,  9.2257061 ,
        9.21096611,  9.22043228,  9.19632721,  9.20783806,  9.20282078,
        9.19964981,  9.2028656 ,  9.2237215 ,  9.21790695,  9.21510983,
        9.19886875,  9.22486019,  9.20293617,  9.21429253,  9.22

In [43]:
cost = tf.reduce_sum(loss) / batch_size

session.run(tf.global_variables_initializer())
session.run(cost, feed_dict)


184.20708

Now, lets store the new state as final state 

In [44]:
# 
final_state = new_state

### Training

To do gradient clipping in TensorFlow we have to take the following steps:

1. Define the optimizer.
2. Extract variables that are trainable.
3. Calculate the gradients based on the loss function.
4. Apply the optimizer to the variables / gradients tuple.

#### 1. Define Optimizer

__GradientDescentOptimizer__ constructs a new gradient descent optimizer. Later, we use constructed __optimizer__ to compute gradients for a loss and apply gradients to variables.

In [45]:
# Create a variable for the learning rate
lr = tf.Variable(0.0, trainable=False)
# Create the gradient descent optimizer with our learning rate
optimizer = tf.train.GradientDescentOptimizer(lr)


#### 2. Trainable Variables

Definining a variable, if you passed _trainable=True_, the _Variable()_ constructor automatically adds new variables to the graph collection __GraphKeys.TRAINABLE_VARIABLES__. Now, using _tf.trainable_variables()_ you can get all variables created with __trainable=True__.

In [46]:
# Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'embedding:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/weights:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/biases:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/weights:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/biases:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

we can find the name and scope of all variables:

In [47]:
tvars=tvars[3:]

In [48]:
[v.name for v in tvars]

['rnn/multi_rnn_cell/cell_1/basic_lstm_cell/weights:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/biases:0',
 'softmax_w:0',
 'softmax_b:0']

#### 3. Calculate the gradients based on the loss function

In [49]:
cost

<tf.Tensor 'truediv:0' shape=() dtype=float32>

In [50]:
tvars

[<tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/weights:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/biases:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

#### Gradient:
The gradient of a function (line) is the slope of the line, or the rate of change of a function. It's a vector (a direction to move) that points in the direction of greatest increase of the function, and calculated by __derivative__ operation.

First lets recall the gradient function using an toy example:
$$ z=\left(2x^2+3xy\right)$$

In [51]:
var_x = tf.placeholder(tf.float32)
var_y = tf.placeholder(tf.float32) 
func_test = 2.0*var_x*var_x + 3.0*var_x*var_y
session.run(tf.global_variables_initializer())
feed={var_x:1.0,var_y:2.0}
session.run(func_test, feed)


8.0

The __tf.gradients()__ function allows you to compute the symbolic gradient of one tensor with respect to one or more other tensors—including variables. __tf.gradients(func,xs)__ constructs symbolic partial derivatives of sum of __func__ w.r.t. _x_ in __xs__. 

Now, lets look at the derivitive w.r.t. __var_x__:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2+3xy\right)=4x+3y $$


In [52]:
var_grad = tf.gradients(func_test, [var_x])
session.run(var_grad,feed)

[10.0]

the derivitive w.r.t. __var_y__:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2+3xy\right)=3x $$

In [53]:
var_grad = tf.gradients(func_test, [var_y])
session.run(var_grad,feed)

[3.0]

Now, we can look at gradients w.r.t all variables:

In [54]:
tf.gradients(cost, tvars)

[<tf.Tensor 'gradients_2/rnn/while/multi_rnn_cell/cell_1/basic_lstm_cell/basic_lstm_cell_1/MatMul/Enter_grad/b_acc_3:0' shape=(400, 800) dtype=float32>,
 <tf.Tensor 'gradients_2/rnn/while/multi_rnn_cell/cell_1/basic_lstm_cell/basic_lstm_cell_1/BiasAdd/Enter_grad/b_acc_3:0' shape=(800,) dtype=float32>,
 <tf.Tensor 'gradients_2/MatMul_grad/MatMul_1:0' shape=(200, 10000) dtype=float32>,
 <tf.Tensor 'gradients_2/add_grad/Reshape_1:0' shape=(10000,) dtype=float32>]

In [55]:
grad_t_list = tf.gradients(cost, tvars)
#sess.run(grad_t_list,feed_dict)




now, we have a list of tensors, t-list. We can use it to find clipped tensors. __clip_by_global_norm__ clips values of multiple tensors by the ratio of the sum of their norms.

__clip_by_global_norm__ get _t-list_ as input and returns 2 things:
 - a list of clipped tensors, so called _list_clipped_ 
 - the global norm (global_norm) of all tensors in t_list

In [56]:
max_grad_norm

5

In [57]:
# Define the gradient clipping threshold
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

[<tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_0:0' shape=(400, 800) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(800,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_2:0' shape=(200, 10000) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_3:0' shape=(10000,) dtype=float32>]

In [58]:
session.run(grads,feed_dict)

[array([[  1.07558318e-08,   9.12204889e-09,   1.12856116e-08, ...,
          -7.70966224e-09,   8.92476137e-09,  -8.08427369e-09],
        [ -6.10435347e-09,   9.43148670e-09,  -1.03863957e-08, ...,
          -1.03984368e-08,   1.23483987e-08,  -8.44202575e-09],
        [ -5.72466208e-09,  -1.07929443e-09,   4.95039343e-09, ...,
          -6.85579460e-09,   8.23519297e-09,   7.05244751e-10],
        ..., 
        [  2.30261255e-09,  -1.12141718e-09,  -1.58951663e-09, ...,
           2.88229907e-09,   6.81134316e-12,   9.39832101e-10],
        [ -1.38669620e-09,  -1.34926359e-09,   1.63711478e-09, ...,
           2.30973085e-09,  -9.56553237e-09,   1.31781841e-09],
        [  1.11014176e-09,  -6.27862784e-09,   2.11078355e-09, ...,
           2.88673974e-09,  -2.77861556e-09,  -5.81890203e-10]], dtype=float32),
 array([ -9.82914344e-07,  -2.70246005e-06,   1.43185162e-06,
         -9.93417899e-08,   3.67005305e-06,   7.76386787e-06,
          3.61585080e-07,   6.92272351e-07,  -9.32212

#### 4. Apply the optimizer to the variables / gradients tuple.

In [59]:
# Create the training TensorFlow Operation through our optimizer
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [60]:
session.run(tf.global_variables_initializer())
session.run(train_op,feed_dict)

We learned how the model is build step by step. Noe, let's then create a Class that represents our model. This class needs a few things:
- We have to create the model in accordance with our defined hyperparameters
- We have to create the placeholders for our input data and expected outputs (the real data)
- We have to create the LSTM cell structure and connect them with our RNN structure
- We have to create the word embeddings and point them to the input data
- We have to create the input structure for our RNN
- We have to instanciate our RNN model and retrieve the variable in which we should expect our outputs to appear
- We need to create a logistic structure to return the probability of our words
- We need to create the loss and cost functions for our optimizer to work, and then create the optimizer
- And finally, we need to create a training operation that can be run to actually train our model


In [61]:
class PTBModel(object):

    def __init__(self, is_training):
        ######################################
        # Setting parameters for ease of use #
        ######################################
        self.batch_size = batch_size
        self.num_steps = num_steps
        size = hidden_size
        self.vocab_size = vocab_size
        
        ###############################################################################
        # Creating placeholders for our input data and expected outputs (target data) #
        ###############################################################################
        self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
        self._targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

        ##########################################################################
        # Creating the LSTM cell structure and connect it with the RNN structure #
        ##########################################################################
        # Create the LSTM unit. 
        # This creates only the structure for the LSTM and has to be associated with a RNN unit still.
        # The argument n_hidden(size=200) of BasicLSTMCell is size of hidden layer, that is, the number of hidden units of the LSTM (inside A).
        # Size is the same as the size of our hidden layer, and no bias is added to the Forget Gate. 
        # LSTM cell processes one word at a time and computes probabilities of the possible continuations of the sentence.
        lstm_cells = []
        reuse = tf.get_variable_scope().reuse
        for _ in range(num_layers):
            cell = tf.contrib.rnn.BasicLSTMCell(size, forget_bias=0.0, reuse=reuse)
            if is_training and keep_prob < 1:
                # Unless you changed keep_prob, this won't actually execute -- this is a dropout wrapper for our LSTM unit
                # This is an optimization of the LSTM output, but is not needed at all
                cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
            lstm_cells.append(cell)
           
        # By taking in the LSTM cells as parameters, the MultiRNNCell function junctions the LSTM units to the RNN units.
        # RNN cell composed sequentially of multiple simple cells.
        stacked_lstm = tf.contrib.rnn.MultiRNNCell(lstm_cells)

        # Define the initial state, i.e., the model state for the very first data point
        # It initialize the state of the LSTM memory. The memory state of the network is initialized with a vector of zeros and gets updated after reading each word.
        self._initial_state = stacked_lstm.zero_state(batch_size, tf.float32)

        ####################################################################
        # Creating the word embeddings and pointing them to the input data #
        ####################################################################
        with tf.device("/cpu:0"):
            # Create the embeddings for our input data. Size is hidden size.
            # Uses default variable initializer
            embedding = tf.get_variable("embedding", [vocab_size, size])  #[10000x200]
            # Define where to get the data for our embeddings from
            inputs = tf.nn.embedding_lookup(embedding, self._input_data)

        # Unless you changed keep_prob, this won't actually execute -- this is a dropout addition for our inputs
        # This is an optimization of the input processing and is not needed at all
        if is_training and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)

        ############################################
        # Creating the input structure for our RNN #
        ############################################
        # Input structure is 20x[30x200]
        # Considering each word is represended by a 200 dimentional vector, and we have 30 batchs, we create 30 word-vectors of size [30xx2000]
        #inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, num_steps, inputs)]
        # The input structure is fed from the embeddings, which are filled in by the input data
        # Feeding a batch of b sentences to a RNN:
        # In step 1,  first word of each of the b sentences (in a batch) is input in parallel.  
        # In step 2,  second word of each of the b sentences is input in parallel. 
        # The parallelism is only for efficiency.  
        # Each sentence in a batch is handled in parallel, but the network sees one word of a sentence at a time and does the computations accordingly. 
        # All the computations involving the words of all sentences in a batch at a given time step are done in parallel. 

        ####################################################################################################
        # Instanciating our RNN model and retrieving the structure for returning the outputs and the state #
        ####################################################################################################
        outputs, state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=self._initial_state)

        #########################################################################
        # Creating a logistic unit to return the probability of the output word #
        #########################################################################
        output = tf.reshape(outputs, [-1, size])
        softmax_w = tf.get_variable("softmax_w", [size, vocab_size]) #[200x1000]
        softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
        logits = tf.matmul(output, softmax_w) + softmax_b

        #########################################################################
        # Defining the loss and cost functions for the model's learning to work #
        #########################################################################
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(self._targets, [-1])],
                                                      [tf.ones([batch_size * num_steps])])
        self._cost = cost = tf.reduce_sum(loss) / batch_size

        # Store the final state
        self._final_state = state

        #Everything after this point is relevant only for training
        if not is_training:
            return

        #################################################
        # Creating the Training Operation for our Model #
        #################################################
        # Create a variable for the learning rate
        self._lr = tf.Variable(0.0, trainable=False)
        # Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
        tvars = tf.trainable_variables()
        # Define the gradient clipping threshold
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), max_grad_norm)
        # Create the gradient descent optimizer with our learning rate
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        # Create the training TensorFlow Operation through our optimizer
        self._train_op = optimizer.apply_gradients(zip(grads, tvars))

    # Helper functions for our LSTM RNN class

    # Assign the learning rate for this model
    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

    # Returns the input data for this model at a point in time
    @property
    def input_data(self):
        return self._input_data

    # Returns the targets for this model at a point in time
    @property
    def targets(self):
        return self._targets

    # Returns the initial state for this model
    @property
    def initial_state(self):
        return self._initial_state

    # Returns the defined Cost
    @property
    def cost(self):
        return self._cost

    # Returns the final state for this model
    @property
    def final_state(self):
        return self._final_state

    # Returns the current learning rate for this model
    @property
    def lr(self):
        return self._lr

    # Returns the training operation defined for this model
    @property
    def train_op(self):
        return self._train_op

With that, the actual structure of our Recurrent Neural Network with Long Short-Term Memory is finished. What remains for us to do is to actually create the methods to run through time -- that is, the `run_epoch` method to be run at each epoch and a `main` script which ties all of this together.

What our `run_epoch` method should do is take our input data and feed it to the relevant operations. This will return at the very least the current result for the cost function.

In [62]:
##########################################################################################################################
# run_epoch takes as parameters the current session, the model instance, the data to be fed, and the operation to be run #
##########################################################################################################################
def run_epoch(session, m, data, eval_op, verbose=False):

    #Define the epoch size based on the length of the data, batch size and the number of steps
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0
    #state = m.initial_state.eval()
    #m.initial_state = tf.convert_to_tensor(m.initial_state) 
    #state = m.initial_state.eval()
    state = session.run(m.initial_state)
    
    #For each step and data point
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
 
        #Evaluate and return cost, state by running cost, final_state and the function passed as parameter
        cost, state, _ = session.run([m.cost, m.final_state, eval_op],
                                     {m.input_data: x,
                                      m.targets: y,
                                      m.initial_state: state})
        
        #Add returned cost to costs (which keeps track of the total costs for this epoch)
        costs += cost
        
        #Add number of steps to iteration counter
        iters += m.num_steps

        if verbose and (step % 10) == 0:
            print("({:.2%})    Perplexity={:.3f}    Speed={:.0f} wps".format(
                    step * 1.0 / epoch_size, 
                    np.exp(costs / iters), 
                    iters * m.batch_size / (time.time() - start_time))
                 )

    # Returns the Perplexity rating for us to keep track of how the model is evolving
    return np.exp(costs / iters)


Now, we create the `main` method to tie everything together. The code here reads the data from the directory, using the `reader` helper module, and then trains and evaluates the model on both a testing and a validating subset of data.

In [63]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _ = raw_data

In [64]:
#Initializes the Execution Graph and the Session
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-init_scale,init_scale)
    
    # Instantiates the model for training
    # tf.variable_scope add a prefix to the variables created with tf.get_variable
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel(is_training=True)
        
    # Reuses the trained parameters for the validation and testing models
    # They are different instances but use the same variables for weights and biases, 
    # they just don't change when data is input
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel(is_training=False)
        mtest = PTBModel(is_training=False)

    #Initialize all variables
    tf.global_variables_initializer().run()
    
    # Set initial learning rate
    m.assign_lr(session=session, lr_value=learning_rate)
    
    for i in range(max_max_epoch):
        print("Epoch %d : Learning rate: %.3f" % (i + 1, session.run(m.lr)))
               
        # Run the loop for this epoch in the training model
        train_perplexity = run_epoch(session, m, train_data, m.train_op,
                                   verbose=True)
        print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
        # Run the loop for this epoch in the validation model
        valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
        
        # Define the decay for the next epoch 
        lr_decay = decay * ((max_max_epoch - i) / max_max_epoch)
        
        # Set the decayed learning rate as the learning rate for the next epoch
        m.assign_lr(session, learning_rate * lr_decay)
    
    # Run the loop in the testing model to see how effective was our training
    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    
    print("Test Perplexity: %.3f" % test_perplexity)

Epoch 1 : Learning rate: 1.000
(0.00%)    Perplexity=10008.760    Speed=2745 wps
(0.65%)    Perplexity=8099.698    Speed=4014 wps
(1.29%)    Perplexity=3874.326    Speed=4117 wps
(1.94%)    Perplexity=2700.296    Speed=4155 wps
(2.58%)    Perplexity=2236.035    Speed=4176 wps
(3.23%)    Perplexity=1956.309    Speed=4186 wps
(3.87%)    Perplexity=1758.261    Speed=4197 wps
(4.52%)    Perplexity=1612.279    Speed=4201 wps
(5.16%)    Perplexity=1504.809    Speed=4208 wps
(5.81%)    Perplexity=1431.562    Speed=4211 wps
(6.46%)    Perplexity=1358.713    Speed=4214 wps
(7.10%)    Perplexity=1299.117    Speed=4216 wps
(7.75%)    Perplexity=1254.090    Speed=4217 wps
(8.39%)    Perplexity=1206.576    Speed=4219 wps
(9.04%)    Perplexity=1168.162    Speed=4220 wps
(9.68%)    Perplexity=1136.567    Speed=4221 wps
(10.33%)    Perplexity=1097.991    Speed=4198 wps
(10.97%)    Perplexity=1075.078    Speed=4180 wps
(11.62%)    Perplexity=1048.018    Speed=4176 wps
(12.27%)    Perplexity=1022.558   

(6.46%)    Perplexity=175.931    Speed=2490 wps
(7.10%)    Perplexity=177.729    Speed=2434 wps
(7.75%)    Perplexity=179.460    Speed=2434 wps
(8.39%)    Perplexity=179.178    Speed=2395 wps
(9.04%)    Perplexity=179.378    Speed=2361 wps
(9.68%)    Perplexity=179.966    Speed=2372 wps
(10.33%)    Perplexity=179.652    Speed=2334 wps
(10.97%)    Perplexity=180.878    Speed=2308 wps
(11.62%)    Perplexity=180.911    Speed=2305 wps
(12.27%)    Perplexity=180.077    Speed=2264 wps
(12.91%)    Perplexity=178.846    Speed=2268 wps
(13.56%)    Perplexity=177.876    Speed=2275 wps
(14.20%)    Perplexity=176.449    Speed=2292 wps
(14.85%)    Perplexity=176.358    Speed=2274 wps
(15.49%)    Perplexity=175.430    Speed=2256 wps
(16.14%)    Perplexity=175.375    Speed=2271 wps
(16.79%)    Perplexity=175.479    Speed=2289 wps
(17.43%)    Perplexity=175.085    Speed=2305 wps
(18.08%)    Perplexity=174.506    Speed=2319 wps
(18.72%)    Perplexity=174.926    Speed=2304 wps
(19.37%)    Perplexity=174

(13.56%)    Perplexity=132.272    Speed=2500 wps
(14.20%)    Perplexity=131.372    Speed=2508 wps
(14.85%)    Perplexity=131.412    Speed=2484 wps
(15.49%)    Perplexity=130.870    Speed=2482 wps
(16.14%)    Perplexity=130.913    Speed=2495 wps
(16.79%)    Perplexity=131.194    Speed=2474 wps
(17.43%)    Perplexity=131.034    Speed=2474 wps
(18.08%)    Perplexity=130.669    Speed=2491 wps
(18.72%)    Perplexity=131.151    Speed=2509 wps
(19.37%)    Perplexity=131.185    Speed=2489 wps
(20.01%)    Perplexity=130.975    Speed=2504 wps
(20.66%)    Perplexity=130.017    Speed=2519 wps
(21.30%)    Perplexity=129.511    Speed=2531 wps
(21.95%)    Perplexity=129.001    Speed=2531 wps
(22.60%)    Perplexity=128.324    Speed=2512 wps
(23.24%)    Perplexity=128.052    Speed=2516 wps
(23.89%)    Perplexity=128.287    Speed=2524 wps
(24.53%)    Perplexity=128.439    Speed=2509 wps
(25.18%)    Perplexity=128.075    Speed=2509 wps
(25.82%)    Perplexity=127.619    Speed=2517 wps
(26.47%)    Perplexi

KeyboardInterrupt: 

As you can see, the model's perplexity rating drops very quickly after a few iterations. As was elaborated before, **lower Perplexity means that the model is more certain about its prediction**. As such, we can be sure that this model is performing well!

-------

This is the end of the **Applying Recurrent Neural Networks to Text Processing** notebook. Hopefully you now have a better understanding of Recurrent Neural Networks and how to implement one utilizing TensorFlow. Thank you for reading this notebook, and good luck on your studies.

Created by <a href="https://br.linkedin.com/in/walter-gomes-de-amorim-junior-624726121">Walter Gomes de Amorim Junior</a>, <a href = "https://linkedin.com/in/saeedaghabozorgi"> Saeed Aghabozorgi </a></h4>